In [ ]:
import sys
import os
import re
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import pandas as pd
import plotly.express as px
import numpy as np

import geopandas as gpd
from pyproj import Transformer
from scripts.utils import split_column_at
from scripts.accent_cleaner import AccentCleaner
#from deep_translator import GoogleTranslator

In [ ]:
df_climate = pd.read_csv("../data/raw/spain_municipalities_climate_final.csv")

In [ ]:
df_climate

In [ ]:
df_climate["Koppen_Climate"].unique()

In [ ]:
climate_translation_dict = {
    'Csa': 'Cfb',
    'Csb': 'Cfa',
    'BSh': 'BSk',
    'Cfa': 'Csa',
    'BWk': 'Bsh',
    'Cwb': 'Csb',
    'Bsk': 'Csa',
}

df_climate['Koppen_Climate'] = df_climate['Koppen_Climate'].map(climate_translation_dict)

In [ ]:
df_climate.loc[df_climate['municipality'] == 'Queralbs', 'Koppen_Climate'] = "Cfb"

In [ ]:
df_climate

In [ ]:
koppen_descriptions = {
    "BWh": "Hot Desert Climate - Dry and very hot.",
    "BWk": "Cold Desert Climate - Dry with large temperature variations.",
    "BSh": "Hot Semi-Arid Climate - Dry with higher temperatures.",
    "BSk": "Cold Semi-Arid Climate - Dry, but with lower temperatures.",
    "Csa": "Mediterranean Hot Summer - Hot, dry summers and mild, wet winters.",
    "Csb": "Mediterranean Warm Summer - Cooler, wet winters and warm, dry summers.",
    "Cfa": "Humid Subtropical - Hot summers with year-round precipitation.",
    "Cfb": "Oceanic Climate - Mild temperatures and consistent rainfall.",
    "Cwb": "Subtropical High Altitude - Moderate temperatures, wet summers."
}

# Convert to DataFrame for merging
df_koppen = pd.DataFrame(list(koppen_descriptions.items()), columns=["Koppen_Climate", "description"])

# Merge climate descriptions with the municipalities dataset
df_climate = df_climate.merge(df_koppen, on="Koppen_Climate", how="left")

In [ ]:
import matplotlib.pyplot as plt

# Count occurrences of each climate type
climate_counts = df_climate['Koppen_Climate'].value_counts()

# Plot the histogram
plt.figure(figsize=(8, 5))
climate_counts.plot(kind='bar')

# Customize the plot
plt.xlabel("Köppen Climate Type")
plt.ylabel("Number of Municipalities")
plt.title("Distribution of Köppen Climate Types")
plt.xticks(rotation=45)  # Rotate x-axis labels if needed
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show plot
plt.show()


In [ ]:

import plotly.express as px

# Create the Interactive Map with Hover Text
fig = px.scatter_mapbox(df_climate, 
                        lat="latitude",  
                        lon="longitude",  
                        color="Koppen_Climate",  
                        hover_data=["Koppen_Climate", "description"],  # Shows climate and description on hover
                        zoom=5,
                        size_max=50,
                        color_discrete_sequence=px.colors.qualitative.Set1  
                        )

# Set map style
fig.update_layout(mapbox_style="open-street-map")

# Show the figure
fig.show()


In [ ]:
df_climate["province"].unique()

In [ ]:
df_climate.to_csv("../data/processed/filtered_climate.csv", index=False)